In [1]:
import os
import pandas as pd
import geopandas as gpd
from scipy import interpolate
from scipy import signal
import ee
import json
from collections import Counter
import datetime
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
# read file using geopandas
gpf = gpd.read_file('merged_irr_final/merged_irr_final/sy_merged_sowing.shp')
gpf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
def get_collection(start_date, end_date, roi, collection='COPERNICUS/S2_SR'):
    """ Filters an ee.ImageCollection. COPERNICUS/S2_SR (Sentinel 2) is the default collection.

        Args:
            start_date (str): start date of the filter (YYYY-MM-DD)
            end_date (str): end date of the filter (YYYY-MM-DD)
            roi (ee.geometry, ee.FeatureCollection): the geometry to intersect
            collection (str): the desired dataset (https://developers.google.com/earth-engine/datasets)

        Returns:
            An ee.ImageCollection
    """
    start = ee.Date(start_date)
    end = ee.Date(end_date)
    filtered_collection = ee.ImageCollection(collection) \
        .filterBounds(roi) \
        .filterDate(start, end)
    return filtered_collection

In [5]:
def calc_s2_vegetation_index(image):
    """ Calculates the vegetation indeces.

        Args:
            image: ee.Image

        Returns:
            Adds the NDVI image to the ee.Image
    """
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [6]:
def cloud_mask_probability(image, max_cloud_prob=65):
    """ Applies the S2 cloud probability mask.

        Args:
            image (ee.Image): ee.Image
            max_cloud_prob: The cloud probability threshold

        Returns:
            Updates the cloud mask in the ee.Image
    """
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(max_cloud_prob)
    return image.updateMask(isNotCloud)

In [7]:
def mask_edges(image):
    """ Sometimes, the mask for the 10m bands does not exclude bad pixels at the image edges.
        Therefore, it's necessary to apply the masks for the 20m and 60m bands as well.
        Reference: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY#description

        Args:
            image (ee.Image): ee.Image to which the mask is to be applied

        Returns:
            Updates the cloud mask in the ee.Image

    """
    return image.updateMask(image.select('B8A').mask().updateMask(image.select('B9').mask()))

In [8]:
def cloud_mask_qa(image):
    """ Applies the S2 QA mask.

Args:
    image (ee.Image): ee.Image

Returns:
    Updates the cloud mask in the ee.Image
    """
    cloudShadowBitMask = (1 << 10)
    cloudsBitMask = (1 << 11)
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

In [9]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=10,
                                  crs='EPSG:3857',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
    def reduce_region_function(img):
        """Applies the ee.Image.reduceRegion() method.
            Reference: https://developers.google.com/earth-engine/tutorials/community/time-series-visualization-with-altair
            Args:
                img:
                    An ee.Image to reduce to a statistic by region.

            Returns:
                An ee.Feature that contains properties representing the image region
                reduction results per band and the image timestamp formatted as
                milliseconds from Unix epoch (included to enable time series plotting)
        """
        try:
            stat = img.reduceRegion(
                reducer=reducer,
                geometry=geometry,#.transform(crs, 0.001)
                scale=scale,
                crs=crs)
                # bestEffort=bestEffort,
                # maxPixels=maxPixels,
                # tileScale=tileScale)
            millis = img.get('system:time_start')
            date_time = ee.Date(millis).format('YYYY-MM-dd HH:mm:ss').getInfo()

            return ee.Feature(geometry, stat).set({ 
                'millis': millis,
                'datetime': date_time,
                'id': img.id()
                }).setGeometry(None)
        except ee.EEException as e:
            # Handle the exception (e.g., print an error message)
            print(f"Error processing image: {e}")
            return None

    return reduce_region_function

In [10]:
def fc_to_dict(fc):
    """ Transfers the properties of the feature to a dictionary.
        Reference: https://developers.google.com/earth-engine/tutorials/community/time-series-visualization-with-altair

        Args:
            fc (ee.FeatureCollection): ee.FeatureCollection

        Returns:
            A dictionary
    """
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
        reducer=ee.Reducer.toList().repeat(prop_names.size()),
        selectors=prop_names).get('list')

    return ee.Dictionary.fromLists(prop_names, prop_lists)

In [13]:
# process input data, get the sowing date and calculate start and end date for the image collection, also calculate irrigation dates
gpf['sowing_dat'] = pd.to_datetime(gpf['sowing'], format='%d-%m-%y').dt.strftime('%Y-%m-%d')
# gpf.head()
# start date is 30 days before sowing date, end date is 150 days after sowing date
gpf['start_date'] = pd.to_datetime(gpf['sowing']) - pd.DateOffset(days=30)
gpf['end_date'] = pd.to_datetime(gpf['sowing']) + pd.DateOffset(days=150)
gpf['sowing_dat'] = pd.to_datetime(gpf['sowing_dat'])
gpf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   plot1       399 non-null    object        
 1   shape_P1    399 non-null    object        
 2   plot2       399 non-null    object        
 3   shape_P2    399 non-null    object        
 4   plot3       399 non-null    object        
 5   shape_P3    399 non-null    object        
 6   sowing      399 non-null    object        
 7   Origin      399 non-null    object        
 8   group       399 non-null    int64         
 9   sowing_dat  399 non-null    datetime64[ns]
 10  mean_ndvi_  399 non-null    float64       
 11  geometry    399 non-null    geometry      
 12  start_date  399 non-null    datetime64[ns]
 13  end_date    399 non-null    datetime64[ns]
dtypes: datetime64[ns](3), float64(1), geometry(1), int64(1), object(8)
memory usage: 43.8+ KB


/var/folders/s0/1w918fdn5qqbxjb9dd551mbw0000gn/T/ipykernel_86114/1575401901.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gpf['start_date'] = pd.to_datetime(gpf['sowing']) - pd.DateOffset(days=30)
/var/folders/s0/1w918fdn5qqbxjb9dd551mbw0000gn/T/ipykernel_86114/1575401901.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gpf['end_date'] = pd.to_datetime(gpf['sowing']) + pd.DateOffset(days=150)


In [16]:
def extract_DAS(row, plot_column):
    das_values = map(int, re.findall(r'(\d+) DAS', row[plot_column]))
    if not das_values:
        return []
    sowing_date = row['sowing_dat']
    irr_dates = [sowing_date + pd.DateOffset(days=das) for das in das_values]
    irr_dates = [date.strftime('%Y-%m-%d') for date in irr_dates]
    # print(type(irr_dates))
    return irr_dates

In [17]:
for index, row in gpf.iterrows():
    if row['Origin'] == 'p1':
        irr_dates1 = extract_DAS(row, 'plot1')
        gpf.at[index, 'irr_dates'] = irr_dates1
    elif row['Origin'] == 'p2':
        irr_dates2 = extract_DAS(row, 'plot2')
        gpf.at[index, 'irr_dates'] = irr_dates2
    elif row['Origin'] == 'p3':
        irr_dates3 = extract_DAS(row, 'plot3')
        gpf.at[index, 'irr_dates'] = irr_dates3

In [18]:
gpf.sample(5)

,plot1,shape_P1,plot2,shape_P2,plot3,shape_P3,sowing,Origin,group,sowing_dat,mean_ndvi_,geometry,start_date,end_date,irr_dates
58,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",83.84666886180639 26.452694873995103,"CT with 2 irrigations (21 DAS, 65 DAS)",83.84684655815363 26.45210263008147,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.84635716100007 26.452765655000064, 83.84651...",14-11-22,p3,20,2022-11-14,0.074324,"POLYGON ((83.84636 26.45277, 83.84651 26.45329...",2022-10-15,2023-04-13,"[2022-12-05, 2023-01-18, 2023-02-07, 2023-02-27]"
365,"CT with 2 irrigations (21 DAS, 65 DAS)","86.24877043068409 24.699862089815788, 86.24885...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.24867051839828 24.700099374111726,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.24854881316423 24.699724105286965,18-12-22,p1,126,2022-12-18,0.092921,"POLYGON ((86.24877 24.69986, 86.24886 24.70004...",2022-11-18,2023-05-17,"[2023-01-08, 2023-02-21]"
205,"CT with 2 irrigations (21 DAS, 65 DAS)",84.83861647546291 24.89153383254241,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",84.83890146017075 24.89154782257033,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","84.83926355838776 24.89154447712903, 84.839254...",11-12-22,p3,72,2022-12-11,0.066999,"POLYGON ((84.83926 24.89154, 84.83925 24.89168...",2022-10-13,2023-04-11,"[2023-01-01, 2023-02-14, 2023-03-06, 2023-03-26]"
291,"CT with 2 irrigations (21 DAS, 65 DAS)",85.57297077029943 25.32546966311181,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",85.57298317551613 25.325457237914705,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","85.57321418076754 25.325703620243488, 85.57321...",20-11-22,p3,101,2022-11-20,0.054474,"POLYGON ((85.57321 25.3257, 85.57321 25.32528,...",2022-10-21,2023-04-19,"[2022-12-11, 2023-01-24, 2023-02-13, 2023-03-05]"
364,"CT with 2 irrigations (21 DAS, 65 DAS)",86.24877043068409 24.699862089815788,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.24867051839828 24.700099374111726, 86.24879...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.24854881316423 24.699724105286965,18-12-22,p2,126,2022-12-18,0.089635,"POLYGON ((86.24867 24.7001, 86.2488 24.70005, ...",2022-11-18,2023-05-17,"[2023-01-08, 2023-02-21, 2023-04-02]"


In [21]:
# Check if geometry is a ee.Geometry
if isinstance(gpf.geometry, gpd.geoseries.GeoSeries):
    gpf['geometry'] = gpf.geometry.apply(lambda x: ee.Geometry.Polygon(x.exterior.coords))
    gpf = gpf.set_geometry('geometry')

In [25]:
type(gpf['geometry'][0])

ee.geometry.Geometry

In [66]:
test = gpf.iloc[0]
test

plot1         CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...
shape_P1      83.27556654810905 26.51913650693408, 83.275912...
plot2          CT with 3 irrigations  (21 DAS, 65 DAS, 105 DAS)
shape_P2      83.275759331882 26.518783706222347, 83.2762552...
plot3                    CT with 2 irrigations (21 DAS, 65 DAS)
shape_P3      83.27629443258047 26.51849600483952, 83.276851...
sowing                                                 21-11-22
Origin                                                       p2
group                                                         0
sowing_dat                                  2022-11-21 00:00:00
mean_ndvi_                                             0.070139
geometry      ee.Geometry({\n  "functionInvocationValue": {\...
start_date                                  2022-10-22 00:00:00
end_date                                    2023-04-20 00:00:00
irr_dates                  [2022-12-12, 2023-01-25, 2023-03-06]
Name: 0, dtype: object

In [78]:
s2_sr = get_collection(test['start_date'], test['end_date'], test['geometry'], collection="COPERNICUS/S2_SR_HARMONIZED")
s2_sr = s2_sr.map(calc_s2_vegetation_index)
s2_sr.first().getInfo()
s2_cloud_prob = get_collection(test['start_date'], test['end_date'], test['geometry'], collection="COPERNICUS/S2_CLOUD_PROBABILITY")
s2_sr_with_cloud_mask = ee.Join.saveFirst('cloud_mask').apply(
        primary=s2_sr.map(mask_edges).map(cloud_mask_qa),
        secondary=s2_cloud_prob,
        condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))
s2_sr_with_cloud_mask.first().getInfo()
imgCollection = ee.ImageCollection(s2_sr_with_cloud_mask).map(cloud_mask_probability)
# imgCollection.first().getInfo()
reducer = ee.Reducer.mean()
reduce_fuction = create_reduce_region_function(geometry=test['geometry'], reducer=reducer, scale=10)
roi_stat = ee.FeatureCollection(imgCollection.map(reduce_fuction))
df_stats = ee.data.computeFeatures({'expression': roi_stat,
                                'fileFormat': 'PANDAS_DATAFRAME'
                                })
df_stats.describe()

,AOT,B1,B11,B12,B2,B3,B4,B5,B6,B7,...,NDVI,QA10,QA20,QA60,SCL,TCI_B,TCI_G,TCI_R,WVP,millis
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,...,22.000000,22.0,22.0,22.0,22.000000,22.000000,22.000000,22.000000,22.000000,3.600000e+01
mean,406.947548,579.347874,2248.036694,1611.412467,752.591350,999.039455,1030.839360,1454.612329,2407.501537,2842.742625,...,0.484046,0.0,0.0,0.0,4.727273,75.433046,100.795072,104.691755,1506.325850,1.673975e+12
std,129.494466,558.078717,813.126493,938.842539,541.099168,502.141160,646.958434,581.139907,700.683521,916.479448,...,0.281579,0.0,0.0,0.0,0.882735,49.391758,47.232921,64.537523,446.598743,4.551402e+09
min,144.772401,0.000000,1396.705929,670.528969,72.718280,420.963059,289.798001,818.918145,1373.655850,1552.452504,...,0.145358,0.0,0.0,0.0,4.000000,7.658282,43.121379,30.219066,832.723557,1.666415e+12
25%,333.000000,285.923254,1533.735712,836.561139,388.880249,619.308977,460.376712,978.344683,1987.042949,2119.977291,...,0.198981,0.0,0.0,0.0,4.000000,40.163541,63.395632,47.300472,1142.564451,1.670195e+12
50%,418.924882,519.759039,1990.728442,1202.430782,642.331597,864.548598,888.865709,1297.130320,2304.275657,2748.514934,...,0.457830,0.0,0.0,0.0,5.000000,66.004110,88.174445,90.766281,1407.460291,1.673975e+12
75%,508.500000,665.640916,3067.768387,2575.614778,1006.442335,1313.949472,1564.182809,1828.101533,2797.196388,3340.054809,...,0.749039,0.0,0.0,0.0,5.000000,102.781627,134.193774,159.516415,1857.033152,1.677755e+12
max,582.000000,2827.574435,4034.177521,3262.571525,2699.054435,2609.962435,2248.223304,2964.783826,4803.013913,5520.078783,...,0.843843,0.0,0.0,0.0,8.000000,240.589043,239.789391,226.662924,2490.663934,1.681535e+12
